# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5649,-52.7093,16.20,93,100,8.23,CA,1691426269
1,1,whitehorse,60.7161,-135.0538,17.43,67,75,4.12,CA,1691426475
2,2,papatowai,-46.5619,169.4708,4.83,89,100,5.26,NZ,1691426475
3,3,cutral-co,-38.9469,-69.2089,9.95,10,75,4.92,AR,1691426236
4,4,west island,-12.1568,96.8225,25.99,83,40,6.69,CC,1691426475


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = .5,
    color = "City",
)

# Display the map plot
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = (city_data_df.loc[(city_data_df["Max Temp"]<28) & (city_data_df["Max Temp"]>22) &
                                (city_data_df["Humidity"]<75) & (city_data_df["Humidity"]>55) &
                                (city_data_df["Cloudiness"]<8)])

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,151,muynoq,43.7683,59.0214,25.48,59,1,2.85,UZ,1691426513
171,171,al burayqah,30.4062,19.5739,27.35,65,0,8.06,LY,1691426517
178,178,lompoc,34.6391,-120.4579,23.08,56,0,1.57,US,1691426519
264,264,chagoda,59.1640,35.3285,25.68,60,0,5.24,RU,1691426538
390,390,maumere,-8.6199,122.2111,23.68,71,2,0.63,ID,1691426570
392,392,jonesboro,35.8423,-90.7043,25.90,65,0,4.47,US,1691426284
418,418,hutchinson,38.0608,-97.9298,25.97,63,0,1.54,US,1691426576
525,525,tobruk,32.0836,23.9764,26.84,67,0,5.89,LY,1691426603
564,564,sapulpa,35.9987,-96.1142,26.61,73,0,1.54,US,1691426612


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
151,muynoq,UZ,43.7683,59.0214,59,
171,al burayqah,LY,30.4062,19.5739,65,
178,lompoc,US,34.6391,-120.4579,56,
264,chagoda,RU,59.1640,35.3285,60,
390,maumere,ID,-8.6199,122.2111,71,
392,jonesboro,US,35.8423,-90.7043,65,
418,hutchinson,US,38.0608,-97.9298,63,
525,tobruk,LY,32.0836,23.9764,67,
564,sapulpa,US,35.9987,-96.1142,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
muynoq - nearest hotel: Lala
al burayqah - nearest hotel: فندق ماس ليبيا
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
chagoda - nearest hotel: No hotel found
maumere - nearest hotel: Bogor
jonesboro - nearest hotel: Park Place Inn
hutchinson - nearest hotel: Fairfield Inn & Suites by Marriott
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
sapulpa - nearest hotel: Holiday Inn Express & Suites Glenpool-Tulsa South, an IHG Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
151,muynoq,UZ,43.7683,59.0214,59,Lala
171,al burayqah,LY,30.4062,19.5739,65,فندق ماس ليبيا
178,lompoc,US,34.6391,-120.4579,56,Embassy Suites by Hilton Lompoc Central Coast
264,chagoda,RU,59.1640,35.3285,60,No hotel found
390,maumere,ID,-8.6199,122.2111,71,Bogor
392,jonesboro,US,35.8423,-90.7043,65,Park Place Inn
418,hutchinson,US,38.0608,-97.9298,63,Fairfield Inn & Suites by Marriott
525,tobruk,LY,32.0836,23.9764,67,فندق زهرة البطنان -طبرق
564,sapulpa,US,35.9987,-96.1142,73,Holiday Inn Express & Suites Glenpool-Tulsa So...


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)